**Ghostbusters: Co-occurence of varied class pairs, in the training and test Dataset**

Setup python libraries

1.   TensorFlow
2.   Keras
3.   sklearn
4.   openCV
5.   Random
6.   NumPy

In [2]:
import tensorflow.compat.v2 as tf
from tensorflow.keras import datasets, layers, models
from sklearn import tree
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
import cv2
import numpy as np
import random

Setup Variables




In [3]:
num_of_letters = 26 # @param {type:"integer"}
num_of_train_images = 2500 # @param {type:"integer"}
num_of_train_images_ub = 200 # @param {type:"integer"}
batch_size = 64 # @param {type:"integer"}
biased_ratio = 32 # @param {type:"integer"}
add_unbiased = 0 # @param {type:"integer"}
test_degree = 3 # @param {type:"integer"}
train_degree = 3 # @param {type:"integer"}
max_num = 2 # @param {type:"integer"}
testOrTrain = 1 # @param {type:"integer"}
times_addition = 0 # @param {type:"integer"}
alphabetOrSpots = 1 # @param {type:"integer"} Alphabets - 1, Spots - 0
f1Values = [0]
f1MaxValues = [0] 

Image Generation Functions 

1. basicText - generates a 64, 64 image with the letter and at the locationmentioned as input.
2. genAlphabetImage - takes the int value of the character as the input, generates the location of the alphabet on the canvas randomly.
3. imageWithDots - generate a 64, 64 image with dots in random locations.
4. resizeImageForFour - resizing function used while combining the 4 alphabet images.
5. genImage - generate the image based on the letter list given as input.

In [4]:
def basicText(text, fontScale, x_direction, y_direction):
  image = np.zeros((64,64))
  # font 
  font = cv2.FONT_HERSHEY_SIMPLEX 
  org = (x_direction,y_direction)  
  color = (255, 255)  
  thickness = 2
  image = cv2.putText(image, text, org, font, fontScale,  
                 color, thickness, cv2.LINE_AA, False)
  return image 

In [5]:
def genAlphabetImage(text):
  val = chr(text)
  x_direction = random.randint(0, 32)
  y_direction = random.randint(28,55)
  image = basicText(val, 1, x_direction, y_direction)
  return image

In [6]:
def imageWithDots():
  image = np.zeros((64,64))
  # font 
  color = (255, 255)  
  thickness = 2
  text = '.'
  font = cv2.FONT_HERSHEY_SIMPLEX 
  for i in range(15):
    x_direction = random.randint(0, 63)
    y_direction = random.randint(0, 63)
    org = (x_direction,y_direction)  
    image = cv2.putText(image, text, org, font, 1,  
                  color, thickness, cv2.LINE_AA, False)
  return image 

In [7]:
def resizeImageForFour(image, num):
  new2 = cv2.resize(image,None,fx=1/num, fy=1/num, interpolation = cv2.INTER_CUBIC)
  new2 = new2.reshape(new2.shape[0], new2.shape[1])
  return new2

In [8]:
def genImage(letterList, numOfTimes):
  finalImage = np.zeros((64,64))
  valList = [[0,0],[0,32],[32,0],[32,32]]
  label = np.zeros(num_of_letters)
  temp = ''
  letterList = list(dict.fromkeys(letterList))
  random.shuffle(valList)
  random.shuffle(letterList)
  for j in range(len(letterList)):
    if alphabetOrSpots == 1:
      img = genAlphabetImage((65 + letterList[j]))
    elif alphabetOrSpots == 0:
      img = imageWithDots()
    label[letterList[j]] = 1
    resizeImage = resizeImageForFour(img, 2)
    finalImage[valList[j][0]:valList[j][0] + 32,valList[j][1]:valList[j][1] + 32] = resizeImage
    temp = temp + chr(65 + letterList[j]) + '_'
  temp = '../pics/' + temp + str(numOfTimes)+'_'+str(testOrTrain)+'.jpg'
  cv2.imwrite('{}'.format(str(temp)), finalImage)
  return finalImage, np.array(label), temp

Dataset Generation functions

In [9]:
def imageGenBasedOnMatrix(matrixDegree, arrayVal):
  images = []
  labels = []
  imageNames = []
  letterList = []
  if matrixDegree == 1:
    for i in range(len(arrayVal)):
      letterList.clear()
      letterList.append(i)
      for j in range(int(arrayVal[i])):
        image, label, imageName = genImage(letterList, j)
        images.append(image)
        labels.append(label)
        imageNames.append(imageName)
  elif matrixDegree == 2:
    for i in range(arrayVal.shape[0]):
      for j in range(arrayVal.shape[1]):
        for k in range(int(arrayVal[i][j])):
          letterList = [i, j]
          random.shuffle(letterList)
          image, label, imageName = genImage(letterList, k)
          images.append(image)
          labels.append(label)
          imageNames.append(imageName)
  elif matrixDegree == 3:
    for i in range(arrayVal.shape[0]):
      for j in range(arrayVal.shape[1]):
        for k in range(arrayVal.shape[2]):
          for l in range(int(arrayVal[i][j][k])):
            letterList = [i,j,k]
            random.shuffle(letterList)
            image, label, imageName = genImage(letterList, l)
            images.append(image)
            labels.append(label)
            imageNames.append(imageName)
  elif matrixDegree == 4:
    for i in range(arrayVal.shape[0]):
      for j in range(arrayVal.shape[1]):
        for k in range(arrayVal.shape[2]):
          for m in range(arrayVal.shape[3]):
            for l in range(int(arrayVal[i][j][k][m])):
              letterList = [i,j,k,m]
              random.shuffle(letterList)
              image, label, imageName = genImage(letterList, l)
              images.append(image)
              labels.append(label)
              imageNames.append(imageName)
  return images, labels, imageNames

In [10]:
def imageGenBasedOnCombinations(comboListVal):
  imageList = []
  labelList = []
  imageNameList = []
  for i in range(len(comboListVal)):
    letterList = []
    for j in range(len(list(comboListVal[i]))):
      letterList.append( ord (list(comboListVal[i])[j]) - 65)
    image, label, imageName = genImage(letterList, 1)
    imageList.append(image)
    labelList.append(label)
    imageNameList.append(imageName)
  return imageList, labelList, imageNameList

In [11]:
def imageGenBasedOnLabels(array, maxValue, numOfUComb):
  imageList = []
  labelList = []
  imageNameList = []
  comboListVal = []
  maxVal = np.amax(array)
  print(maxVal)
  while len(comboListVal) < numOfUComb:
    for i in range(num_of_letters):
      for j in range(i+1, num_of_letters):
        if array[i][j] >= (maxVal - maxValue) and len(comboListVal) < numOfUComb and i != j:
          k=random.randint(0, 25)
          temp = [i, j, k]
          if temp not in comboListVal:
            comboListVal.append([i, j, k])
        elif len(comboListVal) > numOfUComb:
          break
  for i in range(len(comboListVal)):
    letterList = np.array(comboListVal[i])
    image, label, imageName = genImage(letterList, 1)
    imageList.append(image)
    labelList.append(label)
    imageNameList.append(imageName)
  return imageList, labelList, imageNameList

In [12]:
def createOddEven(testImages, testLabels):
  trainImages = []
  trainLabels = []
  for i in range(len(testImages)):
    indexes = []
    for j in range(len(testLabels[i])):
      if testLabels[i][j] == 1:
        indexes.append(j)
    if (indexes[0]%2 == 0 and indexes[1]%2 == 0 and indexes[2]%2 == 0) or  (indexes[0]%2 != 0 and indexes[1]%2 != 0 and indexes[2]%2 != 0):
      trainImages.append(testImages[i])
      trainLabels.append(testLabels[i])
  return trainImages, trainLabels

In [13]:
def createRatioBased(ratio, testImages, testLabels):
  trainImages = []
  trainLabels = []
  for i in range(len(testImages)):
    if i%ratio == 0:
      trainImages.append(testImages[i])
      trainLabels.append(testLabels[i])
  print(len(trainImages))
  return trainImages, trainLabels

In [14]:
def dataSetGen():
  global testOrTrain
  if test_degree == 1:
    testArrayVal = np.array([1]*num_of_letters)
  if test_degree == 2:
    testArrayVal = np.array([[1]*num_of_letters]*num_of_letters)
    for i in range(num_of_letters):
      for j in range(num_of_letters):
        if i == j:
          testArrayVal[i][j] = 0
        if i > j:
          testArrayVal[i][j] = 0
  if test_degree == 3:
    testArrayVal = np.array([[[1]*num_of_letters]*num_of_letters]*num_of_letters)
    for i in range(num_of_letters):
      for j in range(num_of_letters):
        for k in range(num_of_letters):
          if i == j or j == k or i == k:
            testArrayVal[i][j][k] = 0
          if i > j or j > k or i > k:
            testArrayVal[i][j][k] = 0
  imageTesting, testLabels, testImages = imageGenBasedOnMatrix(test_degree, testArrayVal)
  TestDataset = create_dataset(testImages, testLabels, False)
  return TestDataset, testLabels, testImages

Data processing pipeline

1. Functions - 
    - parse_function (takes the input of filename and provides post processed and normalised images)
    - create_dataset (used for generating tf dataset for both training and test images)
2. Input required is the Image names and the data labels
3. TensorFlow Dataset

In [15]:
def parse_function(filename, label):
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=1)
    # Resize it to fixed shapee 
    image_resized = tf.image.resize(image_decoded, [32, 32])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    image_normalized = tf.image.convert_image_dtype(image_normalized, dtype=tf.float32)
    return image_normalized, label

In [16]:
def create_dataset(filenames, labels, is_training=True):    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE )
    dataset = dataset.cache()
    if (is_training):
      dataset = dataset.shuffle(20000, reshuffle_each_iteration=True)
    # Batch the data for multiple steps
    dataset = dataset.batch(batch_size)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE) 
    return dataset

Model used - 

1. Input shape - (32, 32, 1)
2. Activation function used - RELU
3. Last layer activation function - Sigmoid

In [17]:
def modelDef():
  model = models.Sequential()
  model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(32,32,1)))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Conv2D(filters=48, kernel_size=(3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(num_of_letters, activation='sigmoid'))
  model.summary()
  return model

Extra functions - 

1. Accuracy calculation functions
2. F1 score calculation functions
3. Correlationn matrix gen functions
4. Distance matrix gen functions

In [18]:
def Sort(sub_li): 
    l = len(sub_li) 
    for i in range(0, l): 
        for j in range(0, l-i-1): 
            if (sub_li[j][1] > sub_li[j + 1][1]): 
                tempo = sub_li[j] 
                sub_li[j]= sub_li[j + 1] 
                sub_li[j + 1]= tempo 
    return sub_li 

In [19]:
def calculate_labels(pred):
  predict_labels = []
  for i in range(0, len(pred)):
    temp = pred[i]
    val = 0.5
    labels = np.zeros(num_of_letters)
    for j in range(0, num_of_letters):
      if(temp[j] >= val):
        labels[j] = 1
    predict_labels.append(labels)
  return predict_labels

In [20]:
def label_to_alphabet(cal):
  predictedLabels = []
  for i in range(0, len(cal)):
    setList = []
    for j in range(0, num_of_letters):
      if(cal[i][j] == 1):
        temp = chr(j + 65)
        setList.append(temp)
    predictedLabels.append(set(setList))
  return predictedLabels

In [21]:
def f1score(y_test, y_predicted, array):
  global f1Values
  global f1MaxValues
  temp_f1 = 0
  temp_recall = 0
  temp_precision = 0
  temp_acc = 0
  f1Val = []
  distVal = []
  for i in range(len(y_test)):
    temp_f1 = temp_f1 + f1_score(y_test[i], y_predicted[i])
    temp_recall = temp_recall + recall_score(y_test[i], y_predicted[i])
    temp_precision = temp_precision + precision_score(y_test[i], y_predicted[i])
    temp_acc = temp_acc + accuracy_score(y_test[i], y_predicted[i])
    dist_temp = imageValueFunc(array, y_test[i])
    dist_temp_max = imageValueMaxFunc(array, y_test[i])
    f1Val.append([f1_score(y_test[i], y_predicted[i]), dist_temp])
    distVal.append([f1_score(y_test[i], y_predicted[i]), dist_temp_max])
  f1score = temp_f1 / len(y_test)
  precision = temp_precision / len(y_test)
  recall = temp_recall / len(y_test)
  acc = temp_acc / len(y_test)
  f1Values = f1Val
  f1MaxValues = distVal
  return acc, f1score, precision, recall

In [22]:
def accuracyCalc(labelVals, predictedLabelVals):
  Sum = 0 
  accValList = []
  labelsList = []
  for i in range(0, len(labelVals)):
    currentLabelValue = labelVals[i]
    currentPredLabelValue = predictedLabelVals[i]
    setIntersection = currentPredLabelValue.intersection(currentLabelValue)
    x = float(len(setIntersection)/len(currentLabelValue))
    #print(len(currentPredLabelValue), len(currentLabelValue))
    if len(currentLabelValue) == len(currentPredLabelValue) and x == 1:
      labelsList.append(currentLabelValue)
      accValList.append(1)
      Sum = Sum + x
    else:
      labelsList.append(currentLabelValue)
      accValList.append(0)
  Accuracy = Sum / len(labelVals)
  return Accuracy, labelsList, accValList

In [23]:
def accuracyFunc(y_pred, y_test, array):
  y_predicted = calculate_labels(y_pred)
  accuracy, f1_score, precision, recall  = f1score(y_test, y_predicted, array)
  y_predicted = label_to_alphabet(y_predicted)
  y_tested = label_to_alphabet(y_test)
  count, labelList, accValList = accuracyCalc(y_tested, y_predicted)
  print(round(count, 3), round(accuracy, 3), round(f1_score, 3), round(precision, 3), round(recall,3))
  return round(f1_score, 3), labelList, accValList

In [24]:
def sumOfListVals(matrixDegree, arrayValue):
  sumVal = 0
  if matrixDegree == 1:
    for i in range(arrayValue.shape[0]):
      sumVal = sumVal + arrayValue[i]
  elif matrixDegree == 2:
    for i in range(arrayValue.shape[0]):
      for j in range(arrayValue.shape[1]):
        sumVal = sumVal + arrayValue[i][j]
  elif matrixDegree == 3:
    for i in range(arrayValue.shape[0]):
      for j in range(arrayValue.shape[1]):
        for k in range(arrayValue.shape[2]):
          sumVal = sumVal + arrayValue[i][j][k]
  return sumVal

In [25]:
def probabilityGenMatrix(matrixDegree, arrayValue):
  probability = []
  combo = [] 
  sumVal = sumOfListVals(matrixDegree, arrayValue)
  if matrixDegree == 1:
    for i in range(num_of_letters):
      temp = arrayValue[i]/(sumVal)
      probability.append(temp)
      combo.append([chr(65+i)])
  elif matrixDegree == 2:
    for i in range(num_of_letters):
      for j in range(num_of_letters):
        temp = arrayValue[i][j]/(sumVal)
        if(i != j):
          probability.append(temp)
          combo.append([chr(65+i), chr(65+j)])
  elif matrixDegree == 3:
    for i in range(arrayValue.shape[0]):
      for j in range(arrayValue.shape[1]):
        for k in range(arrayValue.shape[2]):
          temp = arrayValue[i][j][k]/(sumVal)
          if not (i == j or j == k or i == k):
            probability.append(temp)
            combo.append([chr(65+i), chr(65+j), chr(65+k)])
  return probability, combo

In [26]:
def createComboList(listValues, probability):
  comboListVal = []
  listVal = Sort(listValues)
  for i in range(len(listVal)):
    if (listVal[i][1] <= probability):
      comboListVal.append(listVal[i][2])
  return comboListVal

In [27]:
def probabilityListAndCorrectness(arrayValue, TestDataset, testLabels):
  probabilityList, comboList = probabilityGenMatrix(test_degree, arrayValue)
  ypred = model.predict(TestDataset)
  acc , combList, correctness = accuracyFunc(ypred, testLabels)
  combinedList = []
  print(arrayValue.shape)
  for i in range(len(probabilityList)):
    #print(combList[i], comboList[i])
    correctVal = correctness[combList.index(set(comboList[i]))]
    combinedList.append([correctVal, probabilityList[i], comboList[i]])
  return acc, probabilityList, correctness, combinedList, combList

In [41]:
def numOfDataInc(trainImages, trainLabels, totalLength):
  global testOrTrain
  trainLabelsNoice = trainLabels
  combinationList = label_to_alphabet(trainLabelsNoice)
  numOfVals = int(totalLength/len(trainLabels)) + 1
  for i in range(numOfVals):
    testOrTrain = i + 2
    _, trainLabelsComb, trainImagesComb = imageGenBasedOnCombinations(combinationList)
    for j in range(len(trainLabelsComb)):
      if len(trainImages) == totalLength:
        break
      trainImages.append(trainImagesComb[j])
      trainLabels.append(trainLabelsComb[j])
  return trainImages, trainLabels

In [29]:
def correlationMatrix(trainLabels):
  if test_degree == 2:
    w = np.vstack(trainLabels)
    arr = np.zeros([num_of_letters,num_of_letters])
    for i in range(0, w.shape[0]):
      for j in range(0, w.shape[1]):
        if(w[i][j] == 1):
          for k in range(j+1, w.shape[1]):
            if(w[i][k] == 1):
              arr[j][k] += 1
              arr[k][j] += 1
  elif test_degree == 1:
    arr = np.zeros((num_of_letters))
    for i in range(len(trainLabels)):
      for j in range(len(trainLabels[i])):
        if trainLabels[i][j] == 1:
          arr[j] = arr[j] + 1
  elif test_degree == 3:
    w = np.vstack(trainLabels)
    arr = np.zeros([num_of_letters,num_of_letters,num_of_letters])
    for i in range(0, w.shape[0]):
      for j in range(0, w.shape[1]):
        if(w[i][j] == 1):
          for k in range(j+1, w.shape[1]):
            if(w[i][k] == 1):
              for l in range(k+1, w.shape[1]):
                if(w[i][l] == 1):
                  arr[j][k][l] += 1
                  arr[k][j][l] += 1
                  arr[j][l][k] += 1
                  arr[k][l][j] += 1
                  arr[l][k][j] += 1
                  arr[l][j][k] += 1
  return arr

In [30]:
def correlationmt(labels):
  arr = np.zeros([num_of_letters,num_of_letters])
  for i in range(len(labels)):
    for j in range(num_of_letters):
      if labels[i][j] == 1:
        for k in range(j+1, num_of_letters):
          if labels[i][k] == 1:
            arr[j][k] = 1
            arr[k][j] = 1
  return arr

In [31]:
def calculateDist(corr):
  dist = np.zeros([num_of_letters,num_of_letters])
  for i in range(num_of_letters):
    for j in range(num_of_letters):
      if i != j:
        if corr[i][j] == 1 and dist[i][j] == 0:
          dist[i][j] = 1
          dist[j][i] = 1
        elif dist[i][j] == 0:
          for k in range(num_of_letters):
            if corr[k][j] == 1 and k != j and k != i:
              #print("i:", i, "j:", j, "k:", k)
              if corr[i][k] == 1:
                temp1 = 2
                if temp1 < dist[i][j] or dist[i][j] == 0:
                  dist[i][j] = int(temp1)
                  dist[j][i] = int(temp1)
              elif dist[i][k] != 0:
                temp = dist[i][k] + 1
                if temp < dist[i][j] or dist[i][j] == 0:
                  dist[i][j] = int(temp)
                  dist[j][i] = int(temp)
              #print(dist[i][j])
  maxVal = np.amax(dist)
  for i in range(num_of_letters):
    for j in range(num_of_letters):
      if dist[i][j] == 0 and i !=j :
        dist[i][j] = maxVal + 1
      elif i == j:
        dist[i][j] = 0 
  return dist

In [32]:
def imageValueMaxFunc(array, label):
  dist = 0
  for i in range(len(label)):
    if label[i] == 1:
      for j in range(i+1, len(label)):
        if label[j] == 1:
          if array[i][j] == 0:
            if dist < 10:
              dist = 10
          else:
            if dist < array[i][j]:
              dist = array[i][j]
  return int(dist)

In [33]:
def imageValueFunc(array, label):
  dist = 0
  for i in range(len(label)):
    if label[i] == 1:
      for j in range(i+1, len(label)):
        if label[j] == 1:
          if array[i][j] == 0:
            dist = dist + 10
          else:
            dist = dist + array[i][j]
  dist = dist / 3 
  return dist

In [34]:
def convertLabelsToSingleAlphabet(alphabet, trainImages, trainLabels):
  trainLabelsNew = np.zeros((len(trainImages)))
  count = 0 
  for i in range(len(trainImages)):
    if trainLabels[i][alphabet] == 1:
      trainLabelsNew[i] = 1
      count = count + 1
  return trainImages, trainLabelsNew, count

In [35]:
def pickRandom(trainImages, trainLabels, numOfComb):
  trainImagesNew = []
  trainLabelsNew = []
  for i in range(numOfComb):
    temp = random.randint(0, 2599)
    if trainImages[temp] not in trainImagesNew:
      trainImagesNew.append(trainImages[temp])
      trainLabelsNew.append(trainLabels[temp])
  return _, trainLabelsNew, trainImagesNew

In [36]:
random.seed(10)
test_degree=3
alphabetOrSpots = 1
testOrTrain = 0
num_of_letters = 26
_, testLabelsVal, testImagesVal = dataSetGen()

In [37]:
random.seed(100)
test_degree=3
alphabetOrSpots = 1
testOrTrain = 1
num_of_letters = 26
_, trainLabelsVal, trainImagesVal = dataSetGen()

In [47]:
random.seed(1000)
biased_ratio = 64
num_of_letters = 26
num_of_train_images = 5000

trainImagesComb, trainLabelsComb = createRatioBased(biased_ratio, trainImagesVal, trainLabelsVal)
trainImages, trainLabels = numOfDataInc(trainImagesComb, trainLabelsComb, num_of_train_images)
TrainDataset = create_dataset(trainImages, trainLabels)
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)

model = modelDef()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, min_delta=0.0002, restore_best_weights=True)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = model.fit(TrainDataset, epochs=400, validation_data=TestDataset, callbacks=[callback], batch_size = batch_size)

ypred = model.predict(TestDataset)
acc , combList, correctness = accuracyFunc(ypred, testLabelsVal, array)

arrayVal = correlationmt(trainLabelsComb)
array = calculateDist(arrayVal)

41
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 48)        13872     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 6, 48)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 4, 4, 64)          27712     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 256)           

In [48]:
print(array)

[[0. 1. 1. 1. 2. 2. 1. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 2. 1. 2. 2. 1. 2. 1.
  2. 1.]
 [1. 0. 1. 2. 2. 1. 2. 2. 2. 1. 2. 2. 2. 2. 1. 2. 2. 1. 1. 2. 2. 2. 2. 1.
  1. 2.]
 [1. 1. 0. 1. 1. 2. 1. 2. 1. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
  2. 2.]
 [1. 2. 1. 0. 1. 2. 2. 1. 1. 2. 2. 1. 2. 2. 2. 1. 2. 1. 1. 2. 2. 1. 2. 2.
  2. 2.]
 [2. 2. 1. 1. 0. 2. 1. 2. 1. 2. 1. 2. 2. 1. 2. 1. 3. 1. 2. 2. 2. 2. 2. 2.
  2. 2.]
 [2. 1. 2. 2. 2. 0. 1. 2. 2. 2. 1. 2. 3. 2. 2. 1. 2. 1. 2. 1. 2. 2. 2. 2.
  1. 2.]
 [1. 2. 1. 2. 1. 1. 0. 1. 1. 2. 2. 1. 2. 2. 2. 1. 2. 1. 2. 2. 1. 2. 1. 1.
  2. 1.]
 [2. 2. 2. 1. 2. 2. 1. 0. 2. 2. 1. 1. 3. 2. 1. 1. 3. 2. 2. 2. 1. 2. 2. 2.
  2. 1.]
 [2. 2. 1. 1. 1. 2. 1. 2. 0. 1. 2. 2. 2. 2. 1. 2. 2. 1. 1. 3. 2. 2. 2. 2.
  2. 2.]
 [2. 1. 1. 2. 2. 2. 2. 2. 1. 0. 1. 1. 2. 2. 1. 2. 2. 2. 1. 2. 2. 1. 2. 2.
  2. 1.]
 [1. 2. 2. 2. 1. 1. 2. 1. 2. 1. 0. 1. 2. 1. 2. 1. 2. 1. 2. 2. 1. 1. 2. 1.
  2. 2.]
 [2. 2. 2. 1. 2. 2. 1. 1. 2. 1. 1. 0. 2. 2. 1. 2. 2. 1. 2. 2. 1. 1. 1. 2.
  2. 2.]
 [2.